In [1]:
from tqdm.notebook import tqdm

In [2]:
import requests
from bs4 import BeautifulSoup

### 1. KOBIS 영화관 입장권 통합 전산망에서 영화 제목 크롤링

총 1,000개의 영화 제목을 수집합니다.

In [122]:
URL = "http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

In [123]:
page = 5
payload = {'curPage':page}

In [124]:
r = requests.get(URL, params = payload)

In [125]:
soup = BeautifulSoup(r.text, "html.parser")

In [126]:
movie_title = set()

In [127]:
for page_num in tqdm(range(100)):
    r = requests.get(URL, params = {"curPage":page_num+1})
    soup = BeautifulSoup(r.text, "html.parser")
    for i in range(10):
        tmp = str(soup.select("#content > div.rst_sch > table > tbody > tr:nth-of-type("+str(i+1)+") > td:nth-of-type(1)")[0].a)
        movie_title.add(tmp.split(">")[1].split("<")[0])

In [154]:
len(movie_title)

998

### 2. 네이버, 다음 영화에서 별점과 댓글 크롤링

In [33]:
import pandas as pd
import time
import random

In [158]:
import copy

In [315]:
Movie = pd.DataFrame(columns = ["Movie_Code", "Title", "Director_Code", "Date", "Time"])
Director = pd.DataFrame(columns = ["Director_Code", "DName", "DAge", "DSex"])
Actor = pd.DataFrame(columns = ["Actor_Code", "AName", "AAge", "ASex"])

Genre = pd.DataFrame(columns = ["Genre"])

User = pd.DataFrame(columns = ["User_ID", "Password", "Age", "Sex"])

Rating = pd.DataFrame(columns = ["Movie_Code", "Rating_Naver", "Num_Naver", "Rating_Daum", "Num_Daum"])
Rating_Comment = pd.DataFrame(columns = ["Movie_Code", "IsNaver", "Comment"])

User_has_Movie = pd.DataFrame(columns = ["User_ID", "Movie_Code"])
Genre_has_Movie = pd.DataFrame(columns = ["Genre", "Movie_Code"])
Movie_has_Actor = pd.DataFrame(columns = ["Movie_Code", "Actor_Code"])

In [316]:
for title in tqdm(movie_title):
    try:
        code = ""
        for i in str(title.encode('euc-kr')).split("\\x")[1:]:
            code += "%"+i.upper().replace(" ","+")
        url = "https://movie.naver.com/movie/search/result.nhn?query=" + code + "&section=all"
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        movie_code = str(soup.select("#old_content > ul.search_list_1 > li > p > a")[0]).split("=")[2][:6]
        movie_code

        # 네이버 고유 영화 코드 찾기
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        movie_code = str(soup.select("#old_content > ul.search_list_1 > li > p > a")[0]).split("=")[2][:6]
        
        # 장르 수집하기
        url = "https://movie.naver.com/movie/bi/mi/basic.nhn?code=" + movie_code
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        tmp_genre = []
        try:
            for i in range(1,10):
                tmp_genre.append(str(soup.select("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-of-type(1) > p > span:nth-of-type(1) > a:nth-of-type("+str(i)+")")[0]).split(">")[1].split("<")[0])
        except:
            pass
                
        # 평점 정보 수집하기
        url = "https://movie.naver.com/movie/bi/mi/point.nhn?code="+movie_code+"&onlyActualPointYn=Y#pointAfterTab"
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        rate = round(float(str(soup.select("#actual_point_tab_inner > div > span > span")[0]).split(":")[1].split("%")[0]) / 10.00, 3)
        people_num = int(soup.select("#graph_area > div.grade_netizen > div.title_area.grade_tit > div.sc_area > span > em")[0].text.replace(",",""))
        
        # 평점이 너무 적은 데이터는 표본에서 제거
        if people_num < 5:
            raise NotImplementedError

        # 상위 3개 리뷰 가져오기
        rate_comment = []
        url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code="+movie_code+"&type=after&onlyActualPointYn=Y&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false"
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        try:
            for i in range(3):
                s = soup.select("body > div > div > div.score_result > ul > li:nth-of-type("+str(i+1)+")")[0]
                top_rate = int(float(str(s.span.span).split(":")[1].split("%")[0])//10)
                top_review = soup.select("#_filtered_ment_"+str(i))[0].text.strip()
                rate_comment.append((top_rate,top_review))
        except:
            pass
        
        # 배우 정보 가져오기
        url = "https://movie.naver.com/movie/bi/mi/detail.nhn?code=" + movie_code
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        actor_name_list = []
        for i in range(8):
            try:
                actor_name = str(soup.select("#content > div.article > div.section_group.section_group_frst > div.obj_section.noline > div > div.lst_people_area.height100 > ul > li:nth-of-type("+str(i+1)+") > p > a")[0]).split('"')[3]
                actor_name_list.append(actor_name)
            except:
                break

        # 감독 정보 가져오기
        url = "https://movie.naver.com/movie/bi/mi/detail.nhn?code=" + movie_code
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        dir_name = str(soup.select("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-of-type(2)")[0]).split(">")[3].split("<")[0]
        if dir_name not in list(Director["DName"]):
            Director.loc[(len(Director))] = [len(Director), dir_name, random.randint(1,100), random.randint(0,1)]

        for i in tmp_genre:
            if i not in list(Genre.Genre):
                Genre.loc[len(Genre)] = [i]
        for i in tmp_genre:
            Genre_has_Movie.loc[len(Genre_has_Movie)] = [i,len(Movie)]
        
        for actor_name in actor_name_list:
            if actor_name not in list(Actor.AName):
                Actor.loc[(len(Actor))] = [len(Actor), actor_name, random.randint(1,100), random.randint(0,1)]
            Movie_has_Actor.loc[len(Movie_has_Actor)] = [len(Movie), int(Actor[Actor.AName == actor_name].Actor_Code)]
        
        tmp = random.random()
        rate2 = round(min(rate+1,10) * tmp + max(rate-1,1) * (1-tmp),2)
        Rating.loc[len(Rating)] = [len(Movie), rate, people_num, rate2, random.randint(1,people_num)]
        for comment in rate_comment:
            Rating_Comment.loc[len(Rating_Comment)] = [len(Movie), 1, comment]
        
        Movie.loc[len(Movie)] = [len(Movie), title,int(Director[Director.DName == dir_name].Director_Code) , "", random.randint(60,180)]
    
    
    except:
        continue

In [317]:
Movie

,Movie_Code,Title,Director_Code,Date,Time
0,0,증언,0,,138
1,1,감쪽같은 그녀,1,,139
2,2,증발,2,,106
3,3,엑설런트 어드벤쳐 3,3,,113
4,4,영하의 바람,4,,130
...,...,...,...,...,...
130,130,애비규환,127,,116
131,131,세인트 아가타,128,,80
132,132,찬실이는 복도 많지,129,,149
133,133,부활: 그 증거,130,,135


In [318]:
Rating

,Movie_Code,Rating_Naver,Num_Naver,Rating_Daum,Num_Daum
0,0.0,9.24,195.0,9.18,53.0
1,1.0,8.82,1820.0,8.26,324.0
2,2.0,8.67,37.0,8.63,36.0
3,3.0,10.00,23.0,9.40,11.0
4,4.0,9.00,63.0,9.66,4.0
...,...,...,...,...,...
130,130.0,8.22,315.0,9.20,146.0
131,131.0,5.20,118.0,4.81,30.0
132,132.0,9.16,753.0,9.50,70.0
133,133.0,9.81,384.0,8.99,181.0


In [319]:
Rating_Comment

,Movie_Code,IsNaver,Comment
0,0,1,"(10, 일단 영상이 아름다워요ㅠㅠ 그게 후반으로 갈수록 초반상황이랑 비교가 되는 ..."
1,0,1,"(10, 새벽에 혼자봤어요 감수성 터지는 영화네요. 전쟁의 슬픔을 간접적으로 잘 느..."
2,0,1,"(10, 유령이 옆에 있다면 같이 사는 법을 배워야 한다. 오랜만에 감동받은 영화...."
3,1,1,"(9, 나문희님은 원래부터 믿고보는 배우였지만, 김수안 배우는 ㄹㅇ 깜짝 놀람ㅠ 연..."
4,1,1,"(9, 괜히 아침부터 봤다가 눈물바다 됨)"
...,...,...,...
361,132,1,"(10, 이 시대에 누구나 공감할수있는 소재의 영화입니다 모처럼 좋은 영화 한편 보..."
362,133,1,"(10, 기독교 영화라고만 하기엔 삶의 실제적인 문제를 다루고 있다. 그 누구도 ..."
363,133,1,"(10, 부활에 대해 깊이 생각하게 하는 영화입니다. 죽음을 이기는 능력은 부활의 ..."
364,133,1,"(10, 인간은 모두 죽는다는 사실을 자주 잊고 사는데 죽음 앞에서 내가 어떻게 살..."


In [320]:
Genre

,Genre
0,드라마
1,전쟁
2,다큐멘터리
3,코미디
4,모험
5,범죄
6,액션
7,뮤지컬
8,멜로/로맨스
9,스릴러


In [321]:
Genre_has_Movie

,Genre,Movie_Code
0,드라마,0
1,전쟁,0
2,드라마,1
3,다큐멘터리,2
4,코미디,3
...,...,...
218,드라마,132
219,멜로/로맨스,132
220,판타지,132
221,다큐멘터리,133


In [322]:
Movie

,Movie_Code,Title,Director_Code,Date,Time
0,0,증언,0,,138
1,1,감쪽같은 그녀,1,,139
2,2,증발,2,,106
3,3,엑설런트 어드벤쳐 3,3,,113
4,4,영하의 바람,4,,130
...,...,...,...,...,...
130,130,애비규환,127,,116
131,131,세인트 아가타,128,,80
132,132,찬실이는 복도 많지,129,,149
133,133,부활: 그 증거,130,,135


In [323]:
Actor

,Actor_Code,AName,AAge,ASex
0,0,알리시아 비칸데르,55,0
1,1,키트 해링턴,86,1
2,2,태런 에저튼,23,0
3,3,콜린 모건,67,0
4,4,도미닉 웨스트,91,1
...,...,...,...,...
794,794,김환진,8,0
795,795,함수정,88,0
796,796,홍소영,80,0
797,797,정미숙,65,0


In [324]:
Movie_has_Actor

,Movie_Code,Actor_Code
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
867,134,795
868,134,796
869,134,761
870,134,797


In [299]:
len(Director)

132

In [293]:
Genre.to_csv("Genre Data.csv", encoding = "utf-8")

In [292]:
Rating.to_csv("Rating Data.csv", encoding = "utf-8")

In [333]:
Movie

,Movie_Code,Title,Director_Code,Date,Time
0,0,증언,0,,138
1,1,감쪽같은 그녀,1,,139
2,2,증발,2,,106
3,3,엑설런트 어드벤쳐 3,3,,113
4,4,영하의 바람,4,,130
...,...,...,...,...,...
130,130,애비규환,127,,116
131,131,세인트 아가타,128,,80
132,132,찬실이는 복도 많지,129,,149
133,133,부활: 그 증거,130,,135


In [338]:
Movie.to_csv("Movie Data.csv", encoding = "utf-8")

In [325]:
Director.to_csv("Director Data.csv", encoding = "utf-8")

In [358]:
for i in range(4):
    print(". ".join(map(str,(Rating_Comment.Comment[i]))).replace(",", "."))

10. 일단 영상이 아름다워요ㅠㅠ 그게 후반으로 갈수록 초반상황이랑 비교가 되는 것도 좋고. 전투장면 같은 것이 없는데도 전쟁의 참혹함에 대해서 한번더 생각할 수 있게 해주는 것 같아요
10. 새벽에 혼자봤어요 감수성 터지는 영화네요. 전쟁의 슬픔을 간접적으로 잘 느낄수있었어요. 근데 상영관이적네여 ㅠㅠ 태론 귀여움
10. 유령이 옆에 있다면 같이 사는 법을 배워야 한다. 오랜만에 감동받은 영화....
9. 나문희님은 원래부터 믿고보는 배우였지만. 김수안 배우는 ㄹㅇ 깜짝 놀람ㅠ 연기 너무 잘해서 눈물 줄줄 흘리고 왔습니다...


In [365]:
for i in range(len(Rating_Comment.Comment)):
    Rating_Comment.Comment[i] = ". ".join(map(str,(Rating_Comment.Comment[i]))).replace(",", ".")

In [367]:
Rating_Comment.to_csv("Rating_Comment Data.csv", encoding = "utf-8")

In [327]:
Genre_has_Movie.to_csv("Genre_has_Movie Data.csv", encoding = "utf-8")

In [328]:
Movie_has_Actor.to_csv("Movie_has_Actor Data.csv", encoding = "utf-8")

In [329]:
Actor.to_csv("Actor Data.csv", encoding = "utf-8")

### 3. User Table, Movie_has_User Table 생성

In [368]:
User = pd.DataFrame(columns = ["User_ID", "Password", "Age", "Sex"])
User_has_Movie = pd.DataFrame(columns = ["User_ID", "Movie_Code"])

In [377]:
name_lst = """0 선우준
0 대유찬
0 도지후
1 문채린
0 현준혁
1 신세은
0 최승현
1 추윤아
0 아주원
0 복지한
0 모동현
0 동지원
0 금승현
0 아도훈
1 랑하율
0 기윤성
1 팽채린
0 환예찬
0 양준성
1 초예나""".split("\n")

In [378]:
print(name_lst)

['0 선우준', '0 대유찬', '0 도지후', '1 문채린', '0 현준혁', '1 신세은', '0 최승현', '1 추윤아', '0 아주원', '0 복지한', '0 모동현', '0 동지원', '0 금승현', '0 아도훈', '1 랑하율', '0 기윤성', '1 팽채린', '0 환예찬', '0 양준성', '1 초예나']


In [384]:
for i in name_lst:
    sex, name = i.split()
    User.loc[len(User)] = [len(User), name, "".join([chr(random.randint(97,122)) for _ in range(6)]), random.randint(10,49), int(sex)]

In [385]:
User

,User_ID,User_Name,Password,Age,Sex
0,0,선우준,bjiurf,35,0
1,1,대유찬,hpzeld,13,0
2,2,도지후,bdaejd,38,0
3,3,문채린,ukxfqv,44,1
4,4,현준혁,lmkhzl,12,0
5,5,신세은,looemm,35,1
6,6,최승현,cbayii,11,0
7,7,추윤아,zcqyce,20,1
8,8,아주원,scqpfz,36,0
9,9,복지한,zaoxjd,45,0


In [389]:
for i in range(20):
    for j in range(len(Movie)):
        if random.random() < 0.3:
            User_has_Movie.loc[len(User_has_Movie)] = [i, j]

In [390]:
User_has_Movie

,User_ID,Movie_Code
0,0,7
1,0,10
2,0,12
3,0,14
4,0,15
...,...,...
836,19,128
837,19,130
838,19,132
839,19,133


In [395]:
User.to_csv("User Data.csv", index = False, encoding = "utf-8")

In [392]:
User_has_Movie.to_csv("User_has_Movie Data.csv", encoding = "utf-8")